<img align="center" width="12%" style="padding-right:10px;" src="../Images/Ccom.png">

# Integrated Seabed Mapping Systems <a href="https://teams.microsoft.com/l/channel/19%3af7b302e1f5b84615a207b9e579d62166%40thread.tacv2/Lab%2520D?groupId=81cb9921-c02d-4e3d-b4d7-9ef6985e82b1&tenantId=d6241893-512d-46dc-8d2b-be47e25f5666"><img src="../Images/help.png"  title="Ask questions on Teams" align="right" width="10%" ></a><br><br>  Lab D: Performance Envelope

In lab D 1 we saw that we need a reference system based on the epoch `t_Tx`. 

<img align="center" width="80%" style="padding-right:10px;" src="../Images/virt_array_pos.jpg">

The virtual array is constructed at the mid point between the Tx location on transmit, and the Rx location on receive. At any single time, the two arrays are actually offset as they cannot be installed at the same place on the hull. Additionally, as these are two separate times, the translation of the entire vessel between those two times should be accounted for.

In real time, as the final navigation solution is not yet available, the convention is to have the stored location of the bottom strike referenced in a coordinate system that is relative to a fixed point in the vessel (the RP not the sonar) as it was at a specific epoch ( almost always the Tx time). That coordinate system is oriented in a local level plane that is aligned along and across the vessel. This is fit for purpose when editing swath data as one normally views the solutions in across track coordinates. That way one can see the shape of successive profiles and perform editing based on their continuity (coherence). Many of the artifacts present in swath data are oriented across the swath (e.g. refraction biases, wobbles), or in the along-track.

For this lab, you will locate the Rx and Tx relative to the RP at that instant using lever arms and instantaneous orientation observations. To account for the along track movement of the RP, the vessel speed and azimuth at that time can be used (and knowing the twtt~ 0.23 seconds). Additionally there will be a small azimuth rotation of the SRF between the two epochs

In [1]:
# ## You may use this to turn a notebook into a function - if you uncomment 
# ## the lines below the notebook Lab_D_1.ipynb will be exported as a function
# ## Note that the funtion originally created a script, thus the confusing name - you may change this 

# import os
# import sys
# from pathlib import Path
# lab_a=Path('../Lab_A/') # Get the path to your Lab_A folder
# sys.path.append(str(lab_a.resolve())) # add the Lab_A folder to the list of paths 
# from mycode.notebook_to_script import notebook_to_script
# notebook_to_script('Lab_D_1.ipynb')

pass

In [10]:
%load_ext autoreload
%autoreload 2

import os
import sys
from pathlib import Path
lab_a=Path('../Lab_A/') # Get the path to your Lab_A folder
sys.path.append(str(lab_a.resolve())) # add the Lab_A folder to the list of paths 

from mycode.ping import Ping
from mycode.position import Position
from mycode.vessel import Vessel
from mycode.SSP import SSP
from mycode.Motion import Motion
from datetime import datetime, timezone, timedelta
from pathlib import Path
import os
import sys
import gsw
import scipy.ndimage
import matplotlib.pyplot as plt
from numpy import pi, sin, cos, log, log10, exp, nan, arctan2
import numpy as np


# The classes and data needed are in the lab A mycode and Data folders respectively

lab_a=Path('../Lab_A/') # Get the path to your Lab_A folder
sys.path.append(str(lab_a.resolve())) # add the Lab_A folder to the list of paths 

# Get the data path
abs_path = os.path.abspath(os.path.curdir)



# By default represent numbers in numpy arrays with four decimals
np.set_printoptions(formatter={'all':lambda x: ' '+str('%.4f'%x)})


# Execute the previous step code
# The classes and data needed are in the lab A mycode and Data folders respectively

lab_a=Path('../Lab_A/') # Get the path to your Lab_A folder
sys.path.append(str(lab_a.resolve())) # add the Lab_A folder to the list of paths 

# Get the data path
abs_path = os.path.abspath(os.path.curdir)

# import of your own classes


# By default represent numbers in numpy arrays with four decimals
np.set_printoptions(formatter={'all':lambda x: ' '+str('%.4f'%x)})

# We need a Vessel class object to store the metadata and geometric data descriptive of the vessel

vessel = Vessel()

# The transmit transducer
vessel.lever_arm_trans = np.array([16.26, -1.75,   4.15]).reshape((3, 1))

# The receive transducer
vessel.lever_arm_rec = np.array([14.82, -2.01,   4.17]).reshape((3, 1))

#The Positioning System
vessel.lever_arm_pos = np.array([-5.73, -0.12, -30.00]).reshape((3, 1))

# The Motion Reference Unit
vessel.lever_arm_mru = np.array([0, 0, 0]).reshape((3, 1))
vessel.wl = -2.59

vessel.metadata["name"]="USNS Henson"
vessel.metadata["owned_by"]="United States Navy"
vessel.metadata["operated_by"]="United States Navy"
vessel.metadata["pos_source"]="NavCom (C-Nav)"
vessel.metadata["sonar"]="Kongsberg EM710"
vessel.metadata["mru"]="Applanix POS/MV 320"
vessel.metadata["loa"]=100

beam_n = 391
# We need a Vessel class object to store the metadata and geometric data descriptive of the vessel


# The transmit transducer
vessel.bias_angles_trans = np.array([0.127*pi/180,1.024*pi/180,   359.957*pi/180]).reshape((3, 1))

# The receive transducer
vessel.bias_angles_rec = np.array([0.101*pi/180, 0.894*pi/180,   0.065*pi/180]).reshape((3, 1))
# positions
pos = Position()
pos.read_jhc_file(str(lab_a)+'/Data/Lab_A_GNSS.txt')
# make sure that there is Cartesian representation of the positions
pos.carto_project('utm','ortho')

# Motion data
motions = Motion()
motions.read_jhc_file(str(lab_a)+'/Data/Lab_A_MRU.txt')

# Sound speed data
sound_speed_profile = SSP()
sound_speed_profile.read_jhc_file(str(lab_a)+'/Data/Lab_A_SVP.txt')
ping = Ping()
ping.read('Data/data_ping_4170.txt')


# Get the index of the beam if interest and print its value
b_index = ping.get_beam_index(beam_n)

t_Tx = ping.tx_time + ping.tx_t_offset_beam[b_index]

t_Rx = t_Tx + timedelta( seconds = ping.twtt[b_index])

att_Tx = motions.get_motion(t_Tx)
att_Rx = motions.get_motion(t_Rx)

R_Tx = motions.get_rot_matrix(t_Tx)
R_Rx = motions.get_rot_matrix(t_Rx)

pos_Tx = pos.get_position(t_Tx)
pos_Rx = pos.get_position(t_Rx)

la_Tx = motions.geo_reference_la(t_Tx, vessel.lever_arm_pos)
la_Rx = motions.geo_reference_la(t_Rx, vessel.lever_arm_pos)

rp_Tx = motions.pos_to_rp(t_Tx, pos_Tx, vessel.lever_arm_pos)
rp_Rx = motions.pos_to_rp(t_Rx, pos_Rx, vessel.lever_arm_pos)

# Assumptions made:
#     Projected coordinate system is fit for purpose 
#     We did not go through North in the interpolation of yaw
#     The difference in direction between ellipsoidal height and orthometric height is negligible i.e.,
#         deflections from the normal are insignificant





ModuleNotFoundError: No module named 'mycode.ping'

---
### D.2.0 Constructing the Virtual Array

We saw in D.1 that we need to construct a virtual array referenced in a coordinate system that is relative to a fixed point in the vessel (the RP not the sonar) as it was at a specific epoch.That coordinate system is oriented in a local level plane that is aligned along and across the vessel.


---
### D.2.1 Determine Baseline Vector from `rp_Tx` to `rp_Rx`

Determine the vector from `rp_Tx` to `rp_Rx` and name the result `base_vec_geo` - this will create a vector centered on the RP at Tx but aligned to the georeferenced frame. Ensure that the result is a 3x1 numpy array

    ALERT: In real time we do not yet know the `rp_Rx`, we can either wait
    until the next position is available (in the case of modern MRUs
    typically less than 1/100 s later, or calculate the vector using speed
    over ground and course over ground at transmit time and the TWTT that is
    observed. The former method is more accurate and leads to an
    imperceptible delay in the visualization of collected data

In [3]:
# D.2.1 Determine Baseline Vector from `rp_Tx` to `rp_Rx`

base_vec_geo = ...

    base_vec_geo = [ -0.5131  -0.7875  -0.1114]

---
### D.2.2 Determine the Course over Ground

Now determine the course over ground in radians between the two positions using this difference vector, remember to be cognizant of the quadrant! You may use the quadrant aware version of the numpy implementation for the arctangent


In [4]:
# D.2.2 Determine the Course over Ground `cog` in radians

# in python numpy arctan is the arc tangent - quadrant aware version is numpy.arctan2()
cog = ...

    cog = -2.5640669972959502

---
### D.2.3 Determine the Drift Angle

Determine the **drift angle** `drift_angle_tx` at transmit, this is the difference between the course over ground and the heading.

    Pet peeve alert (ignore if you like): often the drift angle is incorrectly referred to as the crab angle.
    The crab angle is actually a course correction applied by the navigator to achieve the desired course 
    over ground! the Crab angle should therefore be in the opposite direction of the drift angle, but may be of a 
    different magnitude.
    
Make sure that $\text{drift_angle} \in[-\pi,\pi]$. Hint, make use of while loops

In [37]:
### D.2.3 Determine the Drift Angle `drift_angle`

drift_angle = ...

# Constrain the drift angle to the range [-pi,pi]

...

    drift_angle = -0.03731543969317119

---
### D.2.4 Determine the Change in Heading

Also calculate the change in heading $\Delta \theta$ as `head_delta` - this is the one that determines the new orientation wrt to the Tx epoch reference frame.

As with `drift_angle` make sure that $\text{head_delta} \in[-\pi,\pi]$. 

In [38]:
### D.2.4 Determine the Change in Heading

head_delta = ...

# Constrain the change in heading to the range [-pi,pi]

...


In [ ]:
    head_delta = 0.006934786545932159

---
### D.2.5 Rotate the Baseline Vector from `rp_Tx` to `rp_Rx` in the new Coordinate System

Rotate the difference vector to align with the new reference frame using your pick of the angles just determined.
For this operation we need to retrieve just the rotation matrix for the z-axis.


---
#### D.2.5.1 Add Functions Rx, Ry, Rz and Px,Py, Pz

In the file transform.py in the LabA/mycode folder and Add the *function* `Rx()` to it. This is a function NOT a method i.e., you do not have to create a class definition. The function should take an angle in radians as an argument and return the 3D roll rotation matrix for this angle i.e, the rotation around the x-axis. You may use the `Motion.get_rot_matrix()` as inspiration for the implementation of the rotation matrix.

Similarly, add the *function* `Px()` to the `transform.py` file. The function should return a 3D reflection matrix that reflects the X coordinate.

Also add the functions Ry, Py, Rz and Pz for rotations and reflections around the Y, and Z-axis respectively.

#### D.2.5.2 Add the Function swap_xy()

We often need to make the transformation from a global reference frame to a local reference frame in which the x and y axis are swapped e.g., when we transform from Easting, Northings and Up coordinates with an azimuth to Ship's reference frame with X positive forward, Y positive starboard and Z positive down (or in this case a transducer based reference frame that follows the same conventions).  Add the function `swap_xy` to the `transform.py` file to achieve this transformation. This should work for 1x3, 3x1, and 3 vectors

In [39]:
# Example

from mycode.transform import Rz, Pz, Ry, Py, Rx, Px, swap_xy
print( Rz(1))
print(Pz())

vec = np.array([1,2,3])
print(vec)
print( swap_xy(vec))
vec = vec.reshape((1,3))
print(vec.shape)
print(vec)
print( swap_xy(vec))
vec = vec.reshape((3,1))
print(vec.shape)
print( swap_xy(vec))



[[ 0.5403  -0.8415  0.0000]
 [ 0.8415  0.5403  0.0000]
 [ 0.0000  0.0000  1.0000]]
[[ 1.0000  0.0000  0.0000]
 [ 0.0000  1.0000  0.0000]
 [ 0.0000  0.0000  -1.0000]]
[ 1.0000  2.0000  3.0000]
[ 2.0000  1.0000  3.0000]
(1, 3)
[[ 1.0000  2.0000  3.0000]]
[ 2.0000  1.0000  3.0000]
(3, 1)
[[ 2.0000]
 [ 1.0000]
 [ 3.0000]]


Example:

    [[ 0.5403  -0.8415  0.0000]
     [ 0.8415  0.5403  0.0000]
     [ 0.0000  0.0000  1.0000]]
    [[ 1.0000  0.0000  0.0000]
     [ 0.0000  1.0000  0.0000]
     [ 0.0000  0.0000  -1.0000]]
    [ 1.0000  2.0000  3.0000]
    [ 2.0000  1.0000  3.0000]
    (1, 3)
    [[ 1.0000  2.0000  3.0000]]
    [ 2.0000  1.0000  3.0000]
    (3, 1)
    [[ 2.0000]
     [ 1.0000]
     [ 3.0000]]

---
#### D.2.5.3 Align Georeferenced Baseline Vector to align to the Reference frame

Swap the x and y axis of the difference vector `base_vec_geo` and rotate it using the newly added function `Rz()` to align with the new reference frame and form the vector `base_vec_new` (Make sure that you understand what you just did!). Note that the y-coordinate should become close to 0 (you rotate BACK from the cog to 0!) and that the z axis is positive down.

It is up to you to determine which angles should be used for the rotation. **I will start answering questions about this and all further steps no sooner than three days after the assignment is handed out. Feel free to discuss this on Teams - Drawing diagrams will be helpful**

In [1]:
base_vec_new = ...


    base_vec_new = [ 0.9393  -0.0351  0.1114]

---
### D.2.6 Determine the Rotated Transmit-Transducer Lever Arm

Calculate the Rotated Transmit lever arm `lever_arm_tx` in the new coordinate system based on `vessel.lever_arm_trans`, we calculate this by applying roll and pitch rotations, but not heading as it is already aligned to the x direction.

In [2]:
lever_arm_tx = ...

    lever_arm_tx = [[ 16.4017]
                    [ -1.6718]
                    [  3.5863]]


---
### D.2.7 Determine the Rotated and Translated Receive-Transducer Lever Arm

Calculate the Rx lever arm `lever_arm_rx` in the new reference frame - conceptually it is easiest to think of this as a rotation and a translation. Rotate the receive lever arm `vessel.lever_arm_rec` by roll, pitch and the heading difference, then translate the Rx lever arm by the newly aligned difference vector `base_vec_new`. Note that to add the add `base_vec_new` you will need to make it a column vector (you may use the numpy `reshape` method for this). 

In [3]:
lever_arm_rx = ...

    lever_arm_rx = [[ 15.9159]
                    [ -1.8626]
                    [  3.7752]]


---
### D.2.8 Determine the Location of the Virtual Array

We now know the positions of the Tx transducer `lever_arm_tx` and Rx transducer `lever_arm_rx` in the reference frame defined for epoch `t_Tx` - we will position the virtual transducer `lever_arm_vx` as the average position of the two.

In [4]:
lever_arm_vx = ...

    lever_arm_vx = [[ 16.1588]
                    [ -1.7672]
                    [  3.6808]]


---
### D.2.9 Position the Virtual Array Relative to  the Waterline

Remember that `lever_arm_vx` is the location of the Virtual array wrt to the vessel reference frame at epoch `t_Tx`, we want to offset this by the distance of the waterline to the RP at transmit and receive time to be able to relate the virtual transducer positions in the water column (`lever_arm_vx_wl`). This vertical offset is the combination of the static draft and average heave, make sure to get the signs right!

In [5]:
lever_arm_vx_wl = np.zeros((3,1))
lever_arm_vx_wl[0:2,]  = lever_arm_vx[0:2,] #The horizontal position is unaffected
lever_arm_vx_wl[2,0] = ...

NameError: name 'np' is not defined

    lever_arm_vx_wl = [[ 16.1588]
                       [ -1.7672]
                       [  6.3597]]